### Import

In [1]:
import requests
import pymysql
import json
import pandas as pd
import datetime as dt
import pickle
import time
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

In [2]:
import urllib
from urllib.request import Request, urlopen

### DB 사용 예시

In [44]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

sql = "INSERT INTO table_name (value1, value2) VALUES (123, '이름');"
cur.execute(sql)

con.commit()
con.close()


### 환율

#### api 함수

In [57]:
def exchange_rate_api(date):
  API_HOST = "https://www.koreaexim.go.kr/site/program/financial/exchangeJSON"
  key = "?authkey=b60MyMtLUDVQyyiDjZRdFGbKxoJBypkl"
  search = "&searchdate="+date
  data = "&data=AP01"
  
  url = API_HOST+key+search+data
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    response = requests.get(url, headers=headers)
    return response
  except Exception as ex:
    print(ex)
  

#### 데이터 수집

In [58]:
date_range = pd.date_range(start='10/29/2021', end='12/31/2022')

result_list = []
for date in tqdm(date_range):
  if dt.date.weekday(date) in (5, 6) :
    continue
  str_date = str(date).split(" ")[0]
  
  r = json.loads(exchange_rate_api(str_date).text)
  
  flag = True
  for data in r:
    if data["result"] == 4:
      flag = False
      break
    if data["cur_nm"] in ('미국 달러', '일본 옌', '유로') :
      if(data["cur_nm"] == '일본 옌'):
        result_list.append("('{}', '{}', {})".format(str_date, '일본 100엔', data["deal_bas_r"].replace(",", "")))
      else:
        result_list.append("('{}', '{}', {})".format(str_date, data['cur_nm'], data["deal_bas_r"].replace(",", "")))
  if flag == False:
    break

100%|██████████| 429/429 [00:30<00:00, 14.24it/s]


#### 저장 및 불러오기

In [59]:
with open("data(cur_20221231).p", "wb") as f:
  pickle.dump(result_list, f)

In [5]:
with open("data.p", "rb") as f:
  result_list = pickle.load(f)

#### DB 저장

In [60]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [61]:
sql = "INSERT INTO currency(cur_date, cur_name, cur_rate) VALUES " + ",".join(result_list)
cur.execute(sql)
con.close()

### 원자재(석유, 금)

- API가 2020년부터 데이터가 있어서 사용이 불가능.. 하하
- Naver를 통해 크롤링해야할 듯.
  - 주식, 코스닥 코스피도 동일
- 석유 : 경유, 휘발유, 등유 3가지 평균값 활용
  - 네이버 증권에는 등유가 없어서 경유, 휘발유만 사용?
- 금 : 국내금, 국제금 2가지 각각 활용

#### api 함수

##### 오일
- 휘발유 : OIL_GSL
- 경유 : OIL_LO
- 예시 : https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=640

In [ ]:
def oil_api(oil_type):
  page = 641
  end = 15
  oil = "휘발유" if oil_type == "OIL_GSL" else "경유"
  HOST = "https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd={}&page=".format(oil_type)
  
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    result_list = []
    for p in tqdm(range(page, end-1, -1), 
              total = page-end+1, ## 전체 진행수
              desc = 'Desc', ## 진행률 앞쪽 출력 문장
              ncols = 80, ## 진행률 출력 폭 조절
              leave = True, ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음.
            ):
      url = HOST+str(p)
      response = requests.get(url, headers=headers)
      result_list.extend(oil_bs(response, oil))
      
      time.sleep(0.5)
      
    return result_list
  except Exception as ex:
    print("oil_api 오류 발생")
    print(ex)

def oil_bs(response, oil_type):
  start = dt.datetime.strptime("2011-01-01", "%Y-%m-%d")
  end = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
  
  soup = bs(response.text, "html.parser")
  result = soup.select("tbody > tr")
  
  result_list = []  
  try:
    for p in result[::-1]:
      state = p.get("class")[0]
      td_list = p.select("td")
      td = [td_list[0].getText().strip().replace(".", "-"), oil_type, state, td_list[1].getText().strip(), td_list[2].getText().strip(), td_list[3].getText().strip()[1:-1].strip()]
      
      cur_date = dt.datetime.strptime(td[0], "%Y-%m-%d")
      start_diff = (cur_date - start).total_seconds()
      end_diff= (end - cur_date).total_seconds()
      if start_diff < 0 or end_diff <= 0:
        continue
      
      result_list.append("('{}', '{}', '{}', {}, {}, {})".format(td[0], td[1], td[2], td[3].replace(",", ""), td[4], td[5]))
  except Exception as ex:
    print("oil_bs 오류 발생")
    print(ex)
  
  return result_list

##### 금
- 국제금
  - 예시 : https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=CMDT_GC&fdtc=2&page=446
- 국내금
  - 예시 : https://finance.naver.com/marketindex/goldDailyQuote.naver?&page=305



In [93]:
def gold_api(gold_type):
  if gold_type == "national":
    HOST = "https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=CMDT_GC&fdtc=2&page="
    page = 446
    end = 10
    gold = "국제금(달러/트로이온스)"
  else :
    HOST = "https://finance.naver.com/marketindex/goldDailyQuote.naver?&page="
    page = 305
    end = 8
    gold = "국내금(원/g)"
  
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    result_list = []
    for p in tqdm(range(page, end-1, -1), 
              total = page-end+1, ## 전체 진행수
              desc = 'Desc', ## 진행률 앞쪽 출력 문장
              ncols = 80, ## 진행률 출력 폭 조절
              leave = True, ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음.
            ):
      url = HOST+str(p)
      response = requests.get(url, headers=headers)
      result_list.extend(gold_bs(response, gold))
      
      time.sleep(0.5)
      
    return result_list
  except Exception as ex:
    print("gold_api 오류 발생")
    print(ex)

def gold_bs(response, gold_type):
  start = dt.datetime.strptime("2011-01-01", "%Y-%m-%d")
  end = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
  
  soup = bs(response.text, "html.parser")
  result = soup.select("tbody > tr")
  
  result_list = []  
  try:
    for p in result[::-1]:
      state = p.get("class")[0]
      td_list = p.select("td")
      
      td = [td_list[0].getText().strip().replace(".", "-"), gold_type, state, td_list[1].getText().strip(), td_list[2].getText().strip()]
      if gold_type == "국제금(달러/트로이온스)" :
        td.append(td_list[3].getText().strip()[1:-1].strip())
      else :
        temp_value = float(td[-2].replace(",", "")) + (float(td[-1].replace(",", "")) if td[2] == 'down' else (-float(td[-1].replace(",", ""))))
        td.append(str(round(float(td[-1].replace(",", "")) / temp_value * 100, 2)))
      
      cur_date = dt.datetime.strptime(td[0], "%Y-%m-%d")
      start_diff = (cur_date - start).total_seconds()
      end_diff= (end - cur_date).total_seconds()
      if start_diff < 0 or end_diff <= 0:
        continue
      
      result_list.append("('{}', '{}', '{}', {}, {}, {})".format(td[0], td[1], td[2], td[3].replace(",", ""), td[4].replace(",", ""), td[5]))
  except Exception as ex:
    print("gold_bs 오류 발생")
    print(ex)
  
  return result_list

#### 데이터 수집

##### 금

In [50]:
result_list = gold_api("national")

Desc: 100%|███████████████████████████████████| 437/437 [04:29<00:00,  1.62it/s]


In [94]:
result_list = gold_api("daily")

Desc: 100%|███████████████████████████████████| 298/298 [03:10<00:00,  1.57it/s]


#### 저장 및 불러오기

In [95]:
with open("data(domestic_gold_230414).p", "wb") as f:
  pickle.dump(result_list, f)

In [67]:
with open("data(domestic_gold_230414).p", "rb") as f:
  result_list = pickle.load(f)

#### DB 저장

In [96]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [97]:
sql = "INSERT INTO material(material_date, material_name, material_state, material_rate, material_change, material_change_rate) VALUES " + ",".join(result_list)
cur.execute(sql)
con.close()

### 주식 시세!

- 사용 주식 머시깽이들
  - 전기 : 삼성전자(005930), LG전자(066570)
  - 화학 : LG화학(051910), 롯데케미칼(011170)
  - 생명 : 셀트리온(068270), 녹십자(006280)
  - IT : Naver(035420), SK텔레콤(017670)
  - 엔터 : SM엔터테이먼트(041510), JYP엔터테이먼트(035900)

#### api 함수

- 주식 머시깽이들
- 예시 : https://finance.naver.com/item/sise_day.naver?code={}&page=

In [71]:
def stock_api(stock_type):
  page = 304  # 시작 페이지
  end = 8     # 끝 페이지
  
  if stock_type == "035420":
    stock_origin = "네이버"
    stock = "G IT"
  elif stock_type =="005930":
    stock_origin = "삼성전자"
    stock = "A 전자"
  elif stock_type =="051910":
    stock_origin = "LG화학"
    stock = "B 화학"
  elif stock_type =="068270":
    stock_origin = "셀트리온"
    stock = "C 생명"
  elif stock_type =="041510":
    stock_origin = "SM엔터"
    stock = "E 엔터테이먼트"
  elif stock_type =="066570":
    stock_origin = "LG전자"
    stock = "F 전자"
  elif stock_type =="011170":
    stock_origin = "롯데케미칼"
    stock = "G 화학"
  elif stock_type =="006280":
    stock_origin = "녹십자"
    stock = "H 생명"
  elif stock_type =="017670":
    stock_origin = "SK텔레콤"
    stock = "I IT"
  elif stock_type =="035900":
    stock_origin = "JYP엔터"
    stock = "J 엔터테이먼트"
  
  HOST = "https://finance.naver.com/item/sise_day.naver?code={}&page=".format(stock_type)
  
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*", 'User-Agent': 'Mozilla/5.0'}

  try:
    result_list = []
    for p in tqdm(range(page, end-1, -1), 
              total = page-end+1, ## 전체 진행수
              desc = 'Desc', ## 진행률 앞쪽 출력 문장
              ncols = 80, ## 진행률 출력 폭 조절
              leave = True, ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음.
            ):
      url = HOST+str(p)
      req = Request(url, headers=headers)
      with urlopen(req) as response:
        if not response is None:
          result_list.extend(stock_bs(response, stock_origin, stock))
      
      time.sleep(0.5)
      
    return result_list
  except Exception as ex:
    print("stock_api 오류 발생")
    print(ex)

def stock_bs(response, origin, after):
  start = dt.datetime.strptime("2011-01-01", "%Y-%m-%d")
  end = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
  
  soup = bs(response, "html.parser")
  result = soup.select('table > tr[onmouseover="mouseOver(this)"]')
  
  result_list = []  
  try:
    for p in result[::-1]:
      td_list = p.select("td")
      
      if td_list[2].select_one("img") is None:
        state = "same"
      # elif td_list[2].select_one("img")["alt"].strip() == "상승" :
      #   state = "up" 
      # elif td_list[2].select_one("img")["alt"].strip() == "하락" :
      #   state = "down"
      else :
        state = "up" if "up" in td_list[2].select_one("img")["src"] else "down"
      
      td = [
        td_list[0].getText().strip().replace(".", "-"), 
        after,
        origin, 
        state, 
        td_list[1].getText().strip().replace(",", ""), 
        td_list[2].getText().strip().replace(",", ""), 
        td_list[4].getText().strip().replace(",", ""),
        td_list[5].getText().strip().replace(",", ""),
        td_list[6].getText().strip().replace(",", ""),
        ]
      
      if state != "same":
        change_rate = round(int(td[5]) / ( int(td[4]) + (int(td[5]) if state == "down" else (-int(td[5])))) * 100, 1)
        td.append(change_rate)
      else:
        td.append(0)
      
      cur_date = dt.datetime.strptime(td[0], "%Y-%m-%d")
      start_diff = (cur_date - start).total_seconds()
      end_diff= (end - cur_date).total_seconds()
      if start_diff < 0 or end_diff <= 0:
        continue
      
      result_list.append("('{}', '{}', '{}', '{}', {}, {}, {}, {}, {}, {})".format(td[0], td[1], td[2], td[3], td[4], td[5], td[6], td[7], td[8], td[9]))
  except Exception as ex:
    print("stock_bs 오류 발생", td_list[0].getText().strip())
    print(ex)
  
  return result_list

#### 데이터 수집

In [72]:
result_list = stock_api("006280")

Desc: 100%|███████████████████████████████████| 297/297 [03:17<00:00,  1.50it/s]


In [73]:
len(result_list), result_list[0], result_list[-1]

(2957,
 "('2011-01-03', 'H 생명', '녹십자', 'up', 140000, 1000, 140500, 138000, 43583, 0.7)",
 "('2022-12-29', 'H 생명', '녹십자', 'down', 129500, 4000, 133500, 128000, 27725, 3.0)")

#### 저장 및 불러오기

In [74]:
with open("data(stock_녹십자_230420).p", "wb") as f:
  pickle.dump(result_list, f)

In [ ]:
with open("data(stock_temp_230414).p", "rb") as f:
  result_list = pickle.load(f)

#### DB 저장

In [75]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [76]:
sql = "INSERT INTO stock(stock_date, stock_name, stock_name_origin, stock_state, stock_rate, stock_change, stock_low, stock_high, stock_volume, stock_change_rate) VALUES " + ",".join(result_list)
cur.execute(sql)
con.close()

### 주식 뉴스!

- 사용 주식 머시깽이들
  - 전기 : 삼성전자(005930), LG전자(066570)
  - 화학 : LG화학(051910), 롯데케미칼(011170)
  - 생명 : 셀트리온(068270), 녹십자(006280)
  - IT : Naver(035420), SK텔레콤(017670)
  - 엔터 : SM엔터테이먼트(041510), JYP엔터테이먼트(035900)

#### api 함수

- 예시 : https://search.naver.com/search.naver?where=news&query={키워드}&sm=tab_opt&sort=0&photo=3&field=0&pd=3&ds={YYYY.MM.DD}&de={YYYY.MM.DD}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{YYYYMMDD}to{YYYYMMDD}&is_sug_officeid=0

In [85]:
# 수정 : 일별로 최대 5개 추출!
def news_api(news_type):
  if news_type == "네이버":
    after = "G IT"
  elif news_type == "삼성전자":
    after = "A 전자"
  elif news_type =="LG화학":
    after = "B 화학"
  elif news_type =="셀트리온":
    after = "C 생명"
  elif news_type =="SM엔터":
    after = "E 엔터테이먼트"
  elif news_type =="LG전자":
    after = "F 전자"
  elif news_type =="롯데케미칼":
    after = "G 화학"
  elif news_type =="녹십자":
    after = "H 생명"
  elif news_type =="SK텔레콤":
    after = "I IT"
  elif news_type =="JYP엔터":
    after = "J 엔터테이먼트"
    
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*", 'User-Agent': 'Mozilla/5.0'}
  
  result_list = []
  result_list_all = []
  # 검색 결과는 최대 4000개 보여주므로 각 년도의 분기별로 기사 추출.
  st = dt.datetime.now()
  print("시작 시간 : {}".format(st))
  keyword = urllib.parse.quote(news_type)
  for cur_date in pd.date_range(start="1/1/2011", end="12/31/2022"):
    cur = cur_date.strftime("%Y.%m.%d")
    
    try:
      url = "https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=3&field=0&pd=3&ds={date1}&de={date1}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date2}to{date2}&is_sug_officeid=0".format(keyword=keyword, date1=cur, date2=cur.replace(".", ""))
      req = Request(url, headers=headers)
      with urlopen(req) as response:
        if not response is None:
          new_result_list, new_result_list_all = news_bs(response, news_type, after)
          result_list.extend(new_result_list)
          result_list_all.extend(new_result_list_all)

    except Exception as ex:
      print("news_api 오류 발생")
      print(ex)
    finally:
      if cur.split(".")[2] == "01":
        current = dt.datetime.strptime(cur, "%Y.%m.%d")
        past_date = current  - dt.timedelta(days=1)
        if(past_date.year != 2010):
          print(past_date.strftime("%Y.%m"), end=" ")
          if(current.year != past_date.year):
            print()
      time.sleep(0.3)
    
  print()  
  et = dt.datetime.now()
  print("종료 시간 : {}".format(et))
  
  # 소요 시간(초)
  diff = int((et-st).total_seconds())
  
  # 소요 시간 세부 항목 계산
  total_hour = int(diff // 3600)
  diff = int(diff % 3600)
  total_minute = int(diff // 60)
  total_second = int(diff % 60)
  print("소요 시간 : {}시간 {}분 {}초".format(total_hour, total_minute, total_second))
  
  return result_list, result_list_all

def news_bs(response, origin, after):
  soup = bs(response, "lxml")
  result = soup.select("ul.list_news > li.bx")  # 기사 타이틀이 들어있는 태그 리스트
  
  result_list = [] # 결과 저장(기업명 포함 기사만 저장)
  result_list_all = [] # 모든 결과 저장
  try:
    for idx, p in enumerate(result[:5]):
      cur_date = p.select(".info")[2].get_text().strip()[:-1].replace(".", "-")  # 날짜
      cur_title = p.select_one(".news_tit").get_attribute_list("title")[0]  # 타이틀
      cur_title = cur_title.replace("`", '"').replace("'", '"')
      
      result = "('{}', '{}', '{}', '{}')".format(cur_date, origin, after, cur_title)
      if idx < 3 : result_list_all.append(result)
      if origin in cur_title: result_list.append(result)
  except Exception as ex:
    print("news_bs 오류 발생")
    print(ex)
  
  return result_list, result_list_all

##### 이전 코드 : 모든 검색 결과에서 기업명이 들어간 타이틀만 수집

In [3]:
def news_api(news_type):
  if news_type == "네이버":
    stock = "G IT"
    
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*", 'User-Agent': 'Mozilla/5.0'}
  
  result_list = []
  # 검색 결과는 최대 4000개 보여주므로 각 년도의 분기별로 기사 추출.
  st = dt.datetime.now()
  print("시작 시간 : {}".format(st))
  for year in range(2011, 2023):
    print("{}년도".format(year), end=" ")
    for month in range(1, 13):
      nextDate = (dt.datetime(year, month+1, 1) if month < 12 else dt.datetime(year+1, 1, 1))  - dt.timedelta(days=1)
      lastDate = nextDate.strftime("%Y.%m.%d")
      dateList = [('%d.%02d.%02d' %(year, month, 1), '%d.%02d.%02d' %(year, month, 15)),
                  ('%d.%02d.%02d' %(year, month, 16), lastDate)]
      
      keyword = urllib.parse.quote(news_type)
      
      for startDate, endDate in dateList:
        HOST = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=2&photo=3&field=0&pd=3&ds={}&de={}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{}to{},a:all&start=".format(keyword, startDate, endDate, startDate.replace(".", ""), endDate.replace(".", ""))
        
        try:
          page = 1
          max_page = 1
          while True:
            url = HOST + str(page)
            req = Request(url, headers=headers)
            with urlopen(req) as response:
              if not response is None:
                new_result_list, is_next = news_bs(response, news_type, stock)
                result_list.extend(new_result_list)
                
                # 다음 페이지가 있다면 계속 탐색
                if is_next : 
                  page = page + 10
                else :
                  break
            time.sleep(0.1)
          max_page = max(max_page, page)
        except Exception as ex:
          print("news_api 오류 발생")
          print(ex)
      print("{}".format(month), end=" ")
    print("완료! : {}, 최대 페이지 : {}".format(dt.datetime.now(), max_page))
  
  et = dt.datetime.now()
  print("종료 시간 : {}".format(et))
  
  # 소요 시간(초)
  diff = int((et-st).total_seconds())
  
  # 소요 시간 세부 항목 계산
  total_hour = int(diff // 3600)
  diff = int(diff % 3600)
  total_minute = int(diff // 60)
  total_second = int(diff % 60)
  print("소요 시간 : {}시간 {}분 {}초".format(total_hour, total_minute, total_second))
  
  return result_list

def news_bs(response, origin, after):
  soup = bs(response, "lxml")
  result = soup.select("ul.list_news > li.bx")  # 기사 타이틀이 들어있는 태그 리스트
  btn_next = soup.select_one("a.btn_next")  # 다음 페이지 버튼
  
  # 다음 페이지가 있는지 검사
  is_next = False if btn_next.get_attribute_list("href")[0] is None else True
  
  result_list = [] # 결과 저장
  try:
    for p in result:
      cur_date = p.select(".info")[2].get_text().strip()[:-1].replace(".", "-")  # 날짜
      cur_title = p.select_one(".news_tit").get_attribute_list("title")[0]  # 타이틀
      cur_title = cur_title.replace("`", '"').replace("'", '"')
      
      if origin not in cur_title:
        continue
      
      result_list.append("('{}', '{}', '{}', '{}')".format(cur_date, origin, after, cur_title))
  except Exception as ex:
    print("stock_bs 오류 발생")
    print(ex)
  
  return result_list, is_next

In [53]:
# 네이버 2018년도 추출용!
def news_api(news_type):
  if news_type == "네이버":
    stock = "G IT"
    
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*", 'User-Agent': 'Mozilla/5.0'}
  
  result_list = []
  # 검색 결과는 최대 4000개 보여주므로 각 년도의 분기별로 기사 추출.
  st = dt.datetime.now()
  print("시작 시간 : {}".format(st))
  for year in range(2018, 2019):
    print("{}년도".format(year), end=" ")
    for month in range(1, 13):
      nextDate = (dt.datetime(year, month+1, 1) if month < 12 else dt.datetime(year+1, 1, 1))  - dt.timedelta(days=1)  
      lastDate = nextDate.strftime("%Y.%m.%d")
      dateList = [('%d.%02d.%02d' %(year, month, 1), '%d.%02d.%02d' %(year, month, 15)),
                  ('%d.%02d.%02d' %(year, month, 16), lastDate)]
      
      keyword = urllib.parse.quote(news_type)
      
      for startDate, endDate in dateList:
        HOST = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=2&photo=3&field=0&pd=3&ds={}&de={}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{}to{},a:all&start=".format(keyword, startDate, endDate, startDate.replace(".", ""), endDate.replace(".", ""))
        
        try:
          page = 1
          max_page = 1
          while True:
            url = HOST + str(page)
            req = Request(url, headers=headers)
            with urlopen(req) as response:
              if not response is None:
                new_result_list, is_next = news_bs(response, news_type, stock)
                result_list.extend(new_result_list)
                
                # 다음 페이지가 있다면 계속 탐색
                if is_next : 
                  page = page + 10
                else :
                  break
            time.sleep(0.1)
          max_page = max(max_page, page)
        except Exception as ex:
          print("news_api 오류 발생")
          print(ex)
      print("{}".format(month), end=" ")
    print("완료! : {}, 최대 페이지 : {}".format(dt.datetime.now(), max_page))
  
  et = dt.datetime.now()
  print("종료 시간 : {}".format(et))
  
  # 소요 시간(초)
  diff = int((et-st).total_seconds())
  
  # 소요 시간 세부 항목 계산
  total_hour = int(diff // 3600)
  diff = int(diff % 3600)
  total_minute = int(diff // 60)
  total_second = int(diff % 60)
  print("소요 시간 : {}시간 {}분 {}초".format(total_hour, total_minute, total_second))
  
  return result_list

def news_bs(response, origin, after):
  soup = bs(response, "lxml")
  result = soup.select("ul.list_news > li.bx")  # 기사 타이틀이 들어있는 태그 리스트
  btn_next = soup.select_one("a.btn_next")  # 다음 페이지 버튼
  
  # 다음 페이지가 있는지 검사
  is_next = False if btn_next.get_attribute_list("href")[0] is None else True
  
  result_list = [] # 결과 저장
  try:
    for p in result:
      cur_date = p.select(".info")[2].get_text().strip()[:-1].replace(".", "-")  # 날짜
      cur_title = p.select_one(".news_tit").get_attribute_list("title")[0]  # 타이틀
      cur_title = cur_title.replace("`", '"').replace("'", '"')
      
      if origin not in cur_title:
        continue
      
      result_list.append("('{}', '{}', '{}', '{}')".format(cur_date, origin, after, cur_title))
  except Exception as ex:
    print("stock_bs 오류 발생")
    print(ex)
  
  return result_list, is_next

#### 데이터 수집

In [86]:
result_list, result_list_all = news_api("JYP엔터")

시작 시간 : 2023-04-20 15:29:04.486418
2011.01 2011.02 2011.03 2011.04 2011.05 2011.06 2011.07 2011.08 2011.09 2011.10 2011.11 2011.12 
2012.01 2012.02 2012.03 2012.04 2012.05 2012.06 2012.07 2012.08 2012.09 2012.10 2012.11 2012.12 
2013.01 2013.02 2013.03 2013.04 2013.05 2013.06 2013.07 2013.08 2013.09 2013.10 2013.11 2013.12 
2014.01 2014.02 2014.03 2014.04 2014.05 2014.06 2014.07 2014.08 2014.09 2014.10 2014.11 2014.12 
2015.01 2015.02 2015.03 2015.04 2015.05 2015.06 2015.07 2015.08 2015.09 2015.10 2015.11 2015.12 
2016.01 2016.02 2016.03 2016.04 2016.05 2016.06 2016.07 2016.08 2016.09 2016.10 2016.11 2016.12 
2017.01 2017.02 2017.03 2017.04 2017.05 2017.06 2017.07 2017.08 2017.09 2017.10 2017.11 2017.12 
2018.01 2018.02 2018.03 2018.04 2018.05 2018.06 2018.07 2018.08 2018.09 2018.10 2018.11 2018.12 
2019.01 2019.02 2019.03 2019.04 2019.05 2019.06 2019.07 2019.08 2019.09 2019.10 2019.11 2019.12 
2020.01 2020.02 2020.03 2020.04 2020.05 2020.06 2020.07 2020.08 2020.09 2020.10 2020.11 2020

In [87]:
len(result_list), len(result_list_all)

(66, 1364)

#### 저장 및 불러오기

In [81]:
with open("data(news_JYP엔터_230420).p", "wb") as f:
  pickle.dump(result_list, f)

In [6]:
with open("data(news_naver_230417).p", "rb") as f:
  result_list = pickle.load(f)

#### DB 저장

In [82]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

In [83]:
sql = "INSERT INTO news_origin(news_date, news_name_origin, news_name, news_content) VALUES " + ",".join(result_list)
cur.execute(sql)
con.close()